In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import multiprocessing as mp 
import autoPortfolioTree
import curveTreeDB
import portfolio


# In[ ]:
print("STARTING OBJECT DOWNLOAD")


dataObjs = curveTreeDB.getValidModels(params.treeModels, returnEntireObject=True)


# In[ ]:

allModels = []
tickersSeen = []
for item in dataObjs:
    try:
        if item["IS_PROFITABILITY SLIPPAGE"] > 0.51 and item["IS_ANNUALIZED RETURN"] > 0.10:
            model = item["model"]
            print(model.targetTicker, model.getHash(), item["IS_SHARPE SLIPPAGE"], item["IS_SHARPE DIFFERENCE SLIPPAGE"], item["IS_BETA"])
            allModels.append(model)
            if model.targetTicker not in tickersSeen:
                tickersSeen.append(model.targetTicker)
    except:
        continue
        


# In[ ]:

# len(allModels)


# In[ ]:

# len(tickersSeen)


# In[ ]:

import random
factorToTrade = "VTI"#tickersSeen[random.randint(0, len(tickersSeen) - 1)]
# factorToTrade


# In[ ]:

uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData = autoPortfolioTree.computeReturnsForUniqueModelsCache(allModels, factorToTrade)


# In[ ]:

cleanedReturns = modelReturns.fillna(0)
cleanedReturns.columns = [item.getHash() for item in uniqueModels]

cleanedPredictions = modelPredictions.fillna(0)
cleanedPredictions.columns = [item.getHash() for item in uniqueModels]
hashToModel = {}
for item in uniqueModels:
    hashToModel[item.getHash()] = item


# In[ ]:

# cleanedReturns



STARTING OBJECT DOWNLOAD


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/curveTreeDB.py", line 69, in getValidModels
    retrievedModels = list(query.fetch())
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/google/cloud/iterator.py", line 218, in _items_iter
    for page in self._page_iter(increment=False):
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/google/cloud/iterator.py", line 253, in _page_iter
    page = self._next_page()
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/google/cloud/datastore/query.py", line 497, in _next_page
    query=query_pb,
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/google/cloud/datastore/_gax.py", line 134, in run_query
    return super(GAPICDatastoreAPI, self).run_query(*args, **kwargs)
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/google/cloud/gapic/datastore/v1/datastore_client.py", line 258, in run_query
    return self._run

TypeError: must be str, not list

In [ ]:
def storePortfolioInputData(cleanedReturns, cleanedPredictions, hashToModel, joinedData):
    storageClient = storage.Client('money-maker-1236')
    while True:
        try:
            bucket = storageClient.get_bucket(params.validModelsCache)
            blob = storage.Blob(params.validModelsLookup, bucket)
            blob.upload_from_string(pickle.dumps((cleanedReturns, cleanedPredictions, hashToModel, joinedData)))
            print("STORING", organismHash)
            break
        except:
            print("UPLOAD BLOB ERROR:", str(sys.exc_info()))
            time.sleep(10)
    pass

def getPortfolioInputData():
    storageClient = storage.Client('money-maker-1236')
    while True:
        try:
            bucket = storageClient.get_bucket(params.validModelsCache)
            print("ATTEMPTING PULL", params.validModelsLookup)
            blob = storage.Blob(params.validModelsLookup, bucket)
            return pickle.loads(blob.download_as_string())
        except:
            return None
    pass

In [ ]:
##NEED TO STORE

storePortfolioInputData(cleanedReturns, cleanedPredictions, hashToModel, joinedData)